## chlo from 
# European Space Agency's Climate Change Initiative (CCI) programme¶
### https://climate.esa.int/en/projects/ocean-colour/data/

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [6]:
# if acquiring for the first time (1)
ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v5.0-MONTHLY')
#ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-DAILY')
#ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-8DAY')
ds_color

Note:Caching=1


<xarray.Dataset>
Dimensions:             (lat: 4320, lon: 8640, time: 292)
Coordinates:
  * lat                 (lat) float64 89.98 89.94 89.9 ... -89.9 -89.94 -89.98
  * lon                 (lon) float64 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time                (time) datetime64[ns] 1997-09-04 ... 2021-12-01
Data variables: (12/93)
    crs                 (time) int32 ...
    Rrs_412             (time, lat, lon) float32 ...
    Rrs_443             (time, lat, lon) float32 ...
    Rrs_490             (time, lat, lon) float32 ...
    Rrs_510             (time, lat, lon) float32 ...
    Rrs_560             (time, lat, lon) float32 ...
    ...                  ...
    MODISA_nobs_sum     (time, lat, lon) float32 ...
    VIIRS_nobs_sum      (time, lat, lon) float32 ...
    OLCI_nobs_sum       (time, lat, lon) float32 ...
    MERIS_nobs_sum      (time, lat, lon) float32 ...
    SeaWiFS_nobs_sum    (time, lat, lon) float32 ...
    total_nobs_sum      (time, lat, lon) float32 ...
Attributes: (12/49)
    Conventions:                       CF-1.7
    Metadata_Conventions:              Unidata Dataset Discovery v1.0
    cdm_data_type:                     Grid
    comment:                           See summary attribute
    creator_email:                     help@esa-oceancolour-cci.org
    creator_name:                      Plymouth Marine Laboratory
    ...                                ...
    time_coverage_resolution:          P1M
    time_coverage_duration:            P1M
    start_date:                        01-NOV-2021 00:00:00.000000
    stop_date:                         30-NOV-2021 23:59:00.000000
    time_coverage_start:               202111010000Z
    time_coverage_end:                 202111302359Z

In [ ]:
# open CSV
# for each location, select a region +-3 deg, all times
# example
unimak = ds_color['chlor_a'].sel(time = slice('1998-01-01','2019-12-31'),
                                            lat  = slice(55.2, 53.4), 
                                            lon  = slice(-166.7, -163.5)
                                           ).load()
# take a mean in space, and store the time series into a numpy array
# take a mean in space, and store the time series into a numpy array
# save the arrays as dataarray in a dataset
# add locations as other variables 

In [ ]:
# save the file into a netcdf